<a href="https://colab.research.google.com/github/AI4Finance-LLC/ElegantRL/blob/master/eRL_demo_StockTrading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Stock Trading Application in ElegantRL**






# **Part 1: Problem Formulation**
Formally, we model stock trading as a Markov Decision Process (MDP), and formulate the trading objective as maximization of expected return:



*   **State s = [b, p, h]**: a vector that includes the remaining balance b, stock prices p, and stock shares h. p and h are vectors with D dimension, where D denotes the number of stocks. 
*   **Action a**: a vector of actions over D stocks. The allowed actions on each stock include selling, buying, or holding, which result in decreasing, increasing, or no change of the stock shares in h, respectively.
*   **Reward r(s, a, s’)**: The asset value change of taking action a at state s and arriving at new state s’.
*   **Policy π(s)**: The trading strategy at state s, which is a probability distribution of actions at state s.
*   **Q-function Q(s, a)**: the expected return (reward) of taking action a at state s following policy π.
*   **State-transition**: After taking the actions a, the number of shares h is modified, as shown in Fig 3, and the new portfolio is the summation of the balance and the total value of the stocks.

# **Part 2: Stock Trading Environment Design**

**State Space and Action Space**


*   **State Space**: We use a 181-dimensional vector consists of seven parts of information to represent the state space of multiple stocks trading environment: [b, p, h, M, R, C, X], where b is the balance, p is the stock prices, h is the number of shares, M is the Moving Average Convergence Divergence (MACD), R is the Relative Strength Index (RSI), C is the Commodity Channel Index (CCI), and X is the Average Directional Index (ADX).
*   **Action Space**: As a recap, we have three types of actions: selling, buying, and holding for a single stock. We use the negative value for selling, positive value for buying, and zero for holding. In this case, the action space is defined as {-k, …, -1, 0, 1, …, k}, where k is the maximum share to buy or sell in each transaction.


**Easy-to-customize Features**


*   **initial_capital**: the initial capital that the user wants to invest.
*   **tickers**: the stocks that the user wants to trade with.
*   **initial_stocks**: the initial amount of each stock and the default could be zero.
*   **buy_cost_pct, sell_cost_pct**: the transaction fee of each buying or selling transaction.
*   **max_stock**: the user is able to define the maximum number of stocks that are allowed to trade per transaction. Users can also set the maximum percentage of capitals to invest in each stock.
*   **tech_indicator_list**: the list of financial indicators that are taken into account, which is used to define a state.
*   **start_date, start_eval_date, end_eval_date**: the training and backtesting time intervals. Thee time dates (or timestamps) are used, once the training period is specified, the rest is backtesting.


# **Part 3: Install ElegantRL and related packages**

# **Part 4: Import Packages**


*   **ElegantRL**
*   **yfinance**: yfinance aims to solve this problem by offering a reliable, threaded, and Pythonic way to download historical market data from Yahoo! finance.
*   **StockDataFrame**: stockstats inherits and extends pandas.DataFrame to support Stock Statistics and Stock Indicators.



In [1]:
from run import *
from agent import AgentDQN, AgentUADQN
from StockTrading import StockTradingEnv, check_stock_trading_env
import yfinance as yf
from stockstats import StockDataFrame as Sdf

# **Part 5: Specify Agent and Environment**

*   **args.agent**: firstly chooses one DRL algorithm to use from agent.py. In this application, we prefer to choose DDPG and PPO agent.
*   **args.env**: creates the environment, and the user can either customize own environment or preprocess environments from OpenAI Gym and PyBullet Gym from env.py. In this application, we create the self-designed stock trading environment.


> Before finishing initialization of **args**, please see Arguments() in run.py for more details about adjustable hyper-parameters.




In [2]:
# Agent
args = Arguments(if_on_policy=False)
args.agent = AgentDQN()

# Environment
tickers = 'TARA'  # finrl.config.NAS_74_TICKER

tech_indicator_list = [
  'macd', 'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30',
  'close_30_sma', 'close_60_sma']  # finrl.config.TECHNICAL_INDICATORS_LIST

gamma = 0.99
max_stock = 1
initial_capital = 1e6
initial_stocks = 100
buy_cost_pct = 1e-3
sell_cost_pct = 1e-3
start_date = '2008-03-19'
start_eval_date = '2016-01-01'
end_eval_date = '2021-01-01'

args.env = StockTradingEnv('./', gamma, max_stock, initial_capital, buy_cost_pct, 
                           sell_cost_pct, start_date, start_eval_date, 
                           end_eval_date, tickers, tech_indicator_list, 
                           initial_stocks, if_eval=False, if_save=True)
args.env_eval = StockTradingEnv('./', gamma, max_stock, initial_capital, buy_cost_pct, 
                           sell_cost_pct, start_date, start_eval_date, 
                           end_eval_date, tickers, tech_indicator_list, 
                           initial_stocks, if_eval=True, if_save=True)

args.env.target_reward = 3
args.env_eval.target_reward = 3

# Hyperparameters
args.gamma = gamma
args.break_step = 2000
args.net_dim = 2 ** 9
args.max_step = args.env.max_step
args.max_memo = 2000
args.batch_size = 32
args.repeat_times = 4 # repeat_times * target_step == number of times we update before training
args.eval_gap = 2 ** 4
args.eval_times1 = 2 ** 3
args.eval_times2 = 2 ** 5
args.if_allow_break = True
args.rollout_num = 2 # the number of rollout workers (larger is not always faster)
args.target_step = 50 # number of exploration steps before training

# **Part 6: Train and Evaluate the Agent**

> The training and evaluating processes are all finished inside function **train_and_evaluate_mp()**, and the only parameter for it is **args**. It includes the fundamental objects in DRL:

*   agent,
*   environment.

> And it also includes the parameters for training-control:

*   batch_size,
*   target_step,
*   reward_scale,
*   gamma, etc.

> The parameters for evaluation-control:

*   break_step,
*   random_seed, etc.






In [3]:
train_and_evaluate(args) # the training process will terminate once it reaches the target reward.

| GPU id: 0, cwd: ./AgentDQN/StockTradingEnv-v1_0
| Remove history
ID      Step      MaxR |    avgR      stdR       avgQ      avgL |  avgS  stdS
start exploring before training
Begin updating network before training
start training
explore environment
update environment: total_steps=100
0   5.00e+01      0.95 |
0   5.00e+01      0.95 |0.949602  0.000000   0.712099  0.003616 |  1258     0
explore environment
update environment: total_steps=150
explore environment
update environment: total_steps=200
0   1.50e+02      0.96 |
0   1.50e+02      0.96 |0.957582  0.000000   0.929059  0.012766 |  1258     0
explore environment
update environment: total_steps=250
explore environment
update environment: total_steps=300
0   2.50e+02      0.96 |0.889100  0.000000   1.019043  0.013628 |  1258     0
explore environment
update environment: total_steps=350
explore environment
update environment: total_steps=400
explore environment
update environment: total_steps=450
explore environment
update environmen

Understanding the above results::
*   **Step**: the total training steps.
*  **MaxR**: the maximum reward.
*   **avgR**: the average of the rewards.
*   **stdR**: the standard deviation of the rewards.
*   **objA**: the objective function value of Actor Network (Policy Network).
*   **objC**: the objective function value (Q-value)  of Critic Network (Value Network).

# **Part 7: Backtest and Draw the Graph**

In [4]:
args.env_eval.draw_cumulative_return(args, torch)

Loaded act: ./AgentDQN/StockTradingEnv-v1_0
The initial captial is 1000000.0
The initial number of stocks is 100


[0.9983430383941675,
 0.9961179756663353,
 0.9933484826965867,
 0.9914074705297543,
 0.9913837996496709,
 0.989324433082422,
 0.9869336741940066,
 0.9861525351512569,
 0.9836197509823416,
 0.9831936751408418,
 0.9820101311366757,
 0.9824362069781755,
 0.982625574018842,
 0.982696586659092,
 0.9838327889030914,
 0.9832646877810917,
 0.9804715239312598,
 0.9821758272972589,
 0.9814420300146759,
 0.9811816503337594,
 0.9792879799270937,
 0.979453676087677,
 0.9764238034370117,
 0.9742224115892629,
 0.9744827912701794,
 0.9745301330303461,
 0.9743407659896794,
 0.9752402594328458,
 0.9735832978270132,
 0.9742224115892629,
 0.973867348388013,
 0.97327557638593,
 0.9695947545329735,
 0.9684940586090991,
 0.9691095014912654,
 0.9678904511669744,
 0.9687307674099322,
 0.9672158310845997,
 0.9675117170856412,
 0.9675117170856412,
 0.9682691852483075,
 0.969334374852057,
 0.9709558301377645,
 0.9696776026132652,
 0.9705415897363064,
 0.9704705770960564,
 0.9723997538228472,
 0.9721393741419306,


In [5]:
args.env_eval.draw_cumulative_return_while_learning(args, torch)

Loaded act: ./AgentDQN/StockTradingEnv-v1_0
The initial captial is 1000000.0
The initial number of stocks is 100
updating network: 49
updating network: 99
updating network: 149
updating network: 199
updating network: 249
updating network: 299
updating network: 349
updating network: 399
updating network: 449
updating network: 499
updating network: 549
updating network: 599
updating network: 649
updating network: 699
updating network: 749
updating network: 799
updating network: 849
updating network: 899
updating network: 949
updating network: 999
updating network: 1049
updating network: 1099
updating network: 1149
updating network: 1199
updating network: 1249
